In [3]:
import csv
import re

import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
# import eli5

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import zero_one_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

from sklearn.metrics import SCORERS
import wehs_helpers as wh

In [4]:
bosnian_df = pd.read_csv('bosnian.csv',delimiter=';',encoding="utf8")

In [5]:
bosnian_df[:5]

,1. Koliko Vam je godina?,2. Molimo odaberite Vaš rod:,3. Koji stepen obrazovanja posjedujete?,4. Koji je bio Vaš status na tržištu rada PRIJE upisa na kurs?,5. Koji je Vaš TRENUTNI status na tržištu rada?,"6. U kojoj mjeri je pohađanje gore navedenog kursa pomoglo unapređenju Vaše pozicije na tržištu rada, bilo da se radi o pronalasku zaposlenja, boljem poslu, povećanju prihoda (plate) ili sl.",7. Da li Vas je pohađanje ovog kursa motiviralo da u budućnosti uzmete/upišete druge kurseve?,8. Da li je ovo prvi online kurs koji ste ikada pohađali?,9. Da li se identificirate sa barem jednim od navedenih identiteta ili okolnosti :\n- Etnička manjina u sredini u kojoj živim\n- Rom/romkinja\n- Osoba sa invaliditetom\n- Samohrani roditelj\n- Osoba iz ruralne sredine\n- LGBTQ+,Koliko ste zadovoljni stabilnošću i izvjesnošću svoga zaposlenja?,Koliko ste zadovoljni sigurnošću na radu (sigurnost od povrede ili oboljenja na radnom mjestu)?,Koliko ste zadovoljni mogućnostima za karijerno napredovanje i usavršavanje u Vašoj firmi?,Koliko ste zadovoljni finansijskom kompenzacijom (primanjima/platom) za Vaš rad?,"Generalno gledajući, kako biste ocijenili ukupno zadovoljstvo Vašim trenutnim zaposlenjem (radnim mjestom, okruženjem, tretmanom, itd.)?","Ukoliko želite, u ovome polju možete podijeliti svoj generalni utisak o kursu koji ste pohađali te dati preporuke za unapređenje.","11. Molimo Vas da procijenite kvalitet kursa koji ste pohađali. Molimo Vas da se fokusirate samo na kurs koji je pružila kompanija/akademija od koje ste dobili ovaj email. Kvalitet kursa se procjenjuje na skali od 1 do 10, pri čemu 1 predstavlja najniži stepen kvaliteta, a 10 najviši (najbolji) stepen kvaliteta."
0,20-24,Muški,Srednje obrazovanje - četverogodišnja škola,Nezaposlen,Nezaposlen,Nije do sada doprinio poboljšanju moje pozicij...,"Da, kurs me je u značajnoj mjeri motivirao da ...","Da, ovo je prvi kurs (neformalna edukacija) ko...","Da, sa jednim od navedenih",NaN,NaN,NaN,NaN,NaN,NaN,8
1,Više od 45,Muški,Srednje obrazovanje - četverogodišnja škola,Nezaposlen,Nezaposlen,Nije do sada doprinio poboljšanju moje pozicij...,"Da, kurs me je u značajnoj mjeri motivirao da ...","Ne, i ranije sam pohađao/la kurseve",Ne,NaN,NaN,NaN,NaN,NaN,NaN,10
2,20-24,Muški,Osnovno fakultetsko obrazovanje – Bachelor’s d...,Nezaposlen,Nezaposlen,Nije do sada doprinio poboljšanju moje pozicij...,"Da, kurs me je u značajnoj mjeri motivirao da ...","Ne, i ranije sam pohađao/la kurseve",Ne,NaN,NaN,5.0,NaN,NaN,NaN,9
3,25-30,Muški,Osnovno fakultetsko obrazovanje – Bachelor’s d...,Zaposlen/a na puno radno vrijeme,Zaposlen/a na puno radno vrijeme,"Bio je od ogromnog značaja, tj. presudan za pr...","Ne, jer sam pronašao ili unaprijedio zaposlenj...","Da, ovo je prvi kurs (neformalna edukacija) ko...","Da, sa jednim od navedenih",7.0,10.0,8.0,7.0,7.0,NaN,9
4,20-24,Muški,Srednje obrazovanje - četverogodišnja škola,Student,Student,U umjerenoj mjeri je doprinio pronalasku novog...,"Da, kurs me je donekle motivirao da upišem dru...","Da, ovo je prvi kurs (neformalna edukacija) ko...",Ne,NaN,NaN,NaN,NaN,NaN,NaN,8


In [15]:
bosnian_df_transpon=bosnian_df.transpose()

In [16]:
bosnian_df_transpon[:]

,0,1,2,3,4,5
1. Koliko Vam je godina?,20-24,Više od 45,20-24,25-30,20-24,25-30
2. Molimo odaberite Vaš rod:,Muški,Muški,Muški,Muški,Muški,Muški
3. Koji stepen obrazovanja posjedujete?,Srednje obrazovanje - četverogodišnja škola,Srednje obrazovanje - četverogodišnja škola,Osnovno fakultetsko obrazovanje – Bachelor’s d...,Osnovno fakultetsko obrazovanje – Bachelor’s d...,Srednje obrazovanje - četverogodišnja škola,Srednje obrazovanje - četverogodišnja škola
4. Koji je bio Vaš status na tržištu rada PRIJE upisa na kurs?,Nezaposlen,Nezaposlen,Nezaposlen,Zaposlen/a na puno radno vrijeme,Student,Zaposlen/a na puno radno vrijeme
5. Koji je Vaš TRENUTNI status na tržištu rada?,Nezaposlen,Nezaposlen,Nezaposlen,Zaposlen/a na puno radno vrijeme,Student,Zaposlen/a na puno radno vrijeme
"6. U kojoj mjeri je pohađanje gore navedenog kursa pomoglo unapređenju Vaše pozicije na tržištu rada, bilo da se radi o pronalasku zaposlenja, boljem poslu, povećanju prihoda (plate) ili sl.",Nije do sada doprinio poboljšanju moje pozicij...,Nije do sada doprinio poboljšanju moje pozicij...,Nije do sada doprinio poboljšanju moje pozicij...,"Bio je od ogromnog značaja, tj. presudan za pr...",U umjerenoj mjeri je doprinio pronalasku novog...,Nije do sada doprinio poboljšanju moje pozicij...
7. Da li Vas je pohađanje ovog kursa motiviralo da u budućnosti uzmete/upišete druge kurseve?,"Da, kurs me je u značajnoj mjeri motivirao da ...","Da, kurs me je u značajnoj mjeri motivirao da ...","Da, kurs me je u značajnoj mjeri motivirao da ...","Ne, jer sam pronašao ili unaprijedio zaposlenj...","Da, kurs me je donekle motivirao da upišem dru...","Da, kurs me je u značajnoj mjeri motivirao da ..."
8. Da li je ovo prvi online kurs koji ste ikada pohađali?,"Da, ovo je prvi kurs (neformalna edukacija) ko...","Ne, i ranije sam pohađao/la kurseve","Ne, i ranije sam pohađao/la kurseve","Da, ovo je prvi kurs (neformalna edukacija) ko...","Da, ovo je prvi kurs (neformalna edukacija) ko...","Da, ovo je prvi kurs (neformalna edukacija) ko..."
9. Da li se identificirate sa barem jednim od navedenih identiteta ili okolnosti :\n- Etnička manjina u sredini u kojoj živim\n- Rom/romkinja\n- Osoba sa invaliditetom\n- Samohrani roditelj\n- Osoba iz ruralne sredine\n- LGBTQ+,"Da, sa jednim od navedenih",Ne,Ne,"Da, sa jednim od navedenih",Ne,Ne
Koliko ste zadovoljni stabilnošću i izvjesnošću svoga zaposlenja?,NaN,NaN,NaN,7.0,NaN,NaN


In [27]:
bosnian_df_transpon=bosnian_df_transpon.rename(columns={"": "q", 0: "a1", 1: "a2"})

In [18]:
# bosnian_df_transpon['index'] = bosnian_df_transpon.index
bosnian_df_transpon2=bosnian_df_transpon.reset_index()
bosnian_df_transpon2

,index,0,1,2,3,4,5
0,1. Koliko Vam je godina?,20-24,Više od 45,20-24,25-30,20-24,25-30
1,2. Molimo odaberite Vaš rod:,Muški,Muški,Muški,Muški,Muški,Muški
2,3. Koji stepen obrazovanja posjedujete?,Srednje obrazovanje - četverogodišnja škola,Srednje obrazovanje - četverogodišnja škola,Osnovno fakultetsko obrazovanje – Bachelor’s d...,Osnovno fakultetsko obrazovanje – Bachelor’s d...,Srednje obrazovanje - četverogodišnja škola,Srednje obrazovanje - četverogodišnja škola
3,4. Koji je bio Vaš status na tržištu rada PRIJ...,Nezaposlen,Nezaposlen,Nezaposlen,Zaposlen/a na puno radno vrijeme,Student,Zaposlen/a na puno radno vrijeme
4,5. Koji je Vaš TRENUTNI status na tržištu rada?,Nezaposlen,Nezaposlen,Nezaposlen,Zaposlen/a na puno radno vrijeme,Student,Zaposlen/a na puno radno vrijeme
5,6. U kojoj mjeri je pohađanje gore navedenog k...,Nije do sada doprinio poboljšanju moje pozicij...,Nije do sada doprinio poboljšanju moje pozicij...,Nije do sada doprinio poboljšanju moje pozicij...,"Bio je od ogromnog značaja, tj. presudan za pr...",U umjerenoj mjeri je doprinio pronalasku novog...,Nije do sada doprinio poboljšanju moje pozicij...
6,7. Da li Vas je pohađanje ovog kursa motiviral...,"Da, kurs me je u značajnoj mjeri motivirao da ...","Da, kurs me je u značajnoj mjeri motivirao da ...","Da, kurs me je u značajnoj mjeri motivirao da ...","Ne, jer sam pronašao ili unaprijedio zaposlenj...","Da, kurs me je donekle motivirao da upišem dru...","Da, kurs me je u značajnoj mjeri motivirao da ..."
7,8. Da li je ovo prvi online kurs koji ste ikad...,"Da, ovo je prvi kurs (neformalna edukacija) ko...","Ne, i ranije sam pohađao/la kurseve","Ne, i ranije sam pohađao/la kurseve","Da, ovo je prvi kurs (neformalna edukacija) ko...","Da, ovo je prvi kurs (neformalna edukacija) ko...","Da, ovo je prvi kurs (neformalna edukacija) ko..."
8,9. Da li se identificirate sa barem jednim od ...,"Da, sa jednim od navedenih",Ne,Ne,"Da, sa jednim od navedenih",Ne,Ne
9,Koliko ste zadovoljni stabilnošću i izvjesnošć...,NaN,NaN,NaN,7.0,NaN,NaN
